In [ ]:
import anndata as ad
import squidpy as sq
import cellcharter as cc
import pandas as pd
import scanpy as sc
import scvi
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from lightning.pytorch import seed_everything
import seaborn as sns
import pymde

seed_everything(12345)
scvi.settings.seed = 12345

[rank: 0] Seed set to 12345
[rank: 0] Seed set to 12345


In [ ]:
adata_list = []
for sample in ['1', '2', '3', '4', '5', '6', '7']:
    adata = sc.read_10x_h5(filename=f'/data/cephfs-2/unmirrored/projects/lipomap/Xenium_data2/reg{sample}_rang2_ne20/outs/cell_feature_matrix.h5')
    df = pd.read_csv(f'/data/cephfs-2/unmirrored/projects/lipomap/Xenium_data2/reg{sample}_rang2_ne20/outs/cells.csv.gz')
    df.set_index(adata.obs_names, inplace=True)
    adata.obs = df.copy()
    adata.obsm["spatial"] = adata.obs[["x_centroid", "y_centroid"]].copy().to_numpy()
    # adata.obsm['spatial_fov'][:, 1] = np.max(adata.obsm['spatial_fov'][:, 1]) - adata.obsm['spatial_fov'][:, 1] # Flip y axis
    # Remove negative probes
    adata = adata[:, [c for c in adata.var_names if 'NegPrb' not in c]].copy()
    adata.obs['sample'] = sample
    adata_list.append(adata)

Region7_Unsplit = adata_list[6]

Cells_R7=pd.read_csv("/data/cephfs-2/unmirrored/projects/liposarcoma-wgs/Niche_DE_Rang20/20242309_Split_R7_Rang20/Region_7_cells_stats.csv", skiprows=2)
cellIDs_R7=Cells_R7["Cell ID"]
IDs_R7=cellIDs_R7.to_list()
Cells_R8=pd.read_csv("/data/cephfs-2/unmirrored/projects/liposarcoma-wgs/Niche_DE_Rang20/20242309_Split_R7_Rang20/Region_8_cells_stats.csv", skiprows=2)
cellIDs_R8=Cells_R8["Cell ID"]
IDs_R8=cellIDs_R8.to_list()
Cells_R9=pd.read_csv("/data/cephfs-2/unmirrored/projects/liposarcoma-wgs/Niche_DE_Rang20/20242309_Split_R7_Rang20/Region_9_cells_stats.csv", skiprows=2)
cellIDs_R9=Cells_R9["Cell ID"]
IDs_R9=cellIDs_R9.to_list()

Region7_Split=Region7_Unsplit[IDs_R7,:]
Region8_Split=Region7_Unsplit[IDs_R8,:]
Region8_Split.obs["sample"]="8"
Region9_Split=Region7_Unsplit[IDs_R9,:]
Region9_Split.obs["sample"]="9"

adata_list[6]=Region7_Split

adata_list.append(Region8_Split)
adata_list.append(Region9_Split)

adata = ad.concat(adata_list, axis=0, merge='same', pairwise=True, index_unique='_')

# 'spatial_fov' refers to global coordinates of the cells.
# Thus, we are going to use 'spatial_fov' and not 'spatial' as spatial coordinates.
adata.uns['spatial'] = {s: {} for s in adata.obs['sample'].unique()}
adata.obs['sample'] = pd.Categorical(adata.obs['sample'])
adata

sc.pp.filter_genes(adata, min_counts=3)
sc.pp.filter_cells(adata, min_counts=3)
adata.layers["counts"] = adata.X.copy()
sc.pp.normalize_total(adata, target_sum=1e6)
sc.pp.log1p(adata)


# Train SCVI Model

scvi.settings.seed = 12345
scvi.model.SCVI.setup_anndata(
    adata, 
    layer="counts", 
    batch_key='sample'
)
model = scvi.model.SCVI(adata)

model.train(
    max_epochs=19,              # Maximum number of training epochs # alternate try 800
    early_stopping=True,         # Enable early stopping
    early_stopping_patience=45,  # Set patience parameter
    early_stopping_min_delta=0.001,  # Minimum change to qualify as improvement
    enable_progress_bar=True)

model

# Stopped after 585 (Monitored metric elbo_validation did not improve in the last 45 records. Best score: 139.785. Signaling Trainer to stop.)
model.save('/data/cephfs-1/home/users/rauertc_c/liposarcoma-wgs/20241104_CC_Xenium_SCVIEpoch/20241106_400Epoch', overwrite=True)
adata.obsm['X_scVI'] = model.get_latent_representation(adata).astype(np.float32)

SCVI_MDE_KEY = "X_scVI_MDE"
adata.obsm[SCVI_MDE_KEY] = scvi.model.utils.mde(adata.obsm['X_scVI'], accelerator="cpu")

# map RCTD Celltypes to Adata

Name_CT_All= pd.read_csv("/data/cephfs-2/unmirrored/projects/liposarcoma-wgs/20240923cellcharterXenium_auto_k2_k20/NameCT_All.csv", index_col=1)
Name_CT_All = Name_CT_All.drop("Unnamed: 0", axis=1)
Name_CT_All = Name_CT_All.rename(columns={"Ident":"cell_types"})
Name_CT_All = Name_CT_All[~Name_CT_All.index.duplicated(keep='first')]

NameCT_Dict = {Name_CT_All.index[i]:Name_CT_All.cell_types[i] for i in range(len(Name_CT_All.index))}

adata.obs['cell_types'] = adata.obs['cell_id'].map(NameCT_Dict)
adata.obs['cell_types'] = adata.obs['cell_types'].astype('category')

# Plot Embedding
sc.pl.embedding(
    adata,
    basis=SCVI_MDE_KEY,
    color=["sample", "cell_types"],
    frameon=False,
    ncols=1,
    legend_loc="center left",
    legend_fontsize='small',
)

# Get all the axes (one for each subplot) and adjust the legends
fig, axes = plt.gcf(), plt.gcf().axes  # Get current figure and axes

# Adjust legend for each subplot
for ax in axes:
    legend = ax.get_legend()
    legend.set_bbox_to_anchor((1.02, 0.5))  # Shift each legend to the right
    legend._legend_box.align = "left"       # Aligns the content of the legend if needed

# Ensure tight layout to avoid clipping
plt.tight_layout()

# Save the figure with all elements included
plt.savefig('/data/cephfs-1/home/users/rauertc_c/liposarcoma-wgs/20241104_CC_Xenium_SCVIEpoch/20241106_19Epoch/SCVI_19_Embedding.pdf', bbox_inches="tight")
plt.close()

# Plot training and validation loss
plt.plot(model.history['reconstruction_loss_train']['reconstruction_loss_train'], label='train')
plt.plot(model.history['reconstruction_loss_validation']['reconstruction_loss_validation'], label='validation')
plt.legend()
plt.savefig('/data/cephfs-1/home/users/rauertc_c/liposarcoma-wgs/20241104_CC_Xenium_SCVIEpoch/20241106_19Epoch/SCVI_19_TrainValLoss.pdf', bbox_inches="tight")

np.argmin(model.history["reconstruction_loss_validation"])

In [ ]:
# Get Spatial Network & do spatial Clustering
sq.gr.spatial_neighbors(adata, library_key='sample', coord_type='generic', delaunay=True, spatial_key='spatial')
cc.gr.remove_long_links(adata)
cc.gr.aggregate_neighbors(adata, n_layers=3, use_rep='X_scVI', out_key='X_cellcharter', sample_key='sample')

autok = cc.tl.ClusterAutoK(
    n_clusters=(2,20), 
    max_runs=10,
    convergence_tol=0.001
)

autok.fit(adata, use_rep='X_cellcharter')
autok.save('/data/cephfs-1/home/users/rauertc_c/liposarcoma-wgs/20241104_CC_Xenium_SCVIEpoch/20241106_19Epoch')

cc.pl.autok_stability(autok)
plt.savefig('/data/cephfs-1/home/users/rauertc_c/liposarcoma-wgs/20241104_CC_Xenium_SCVIEpoch/20241106_19Epoch/19Epoch_autok_stability.pdf')
plt.close()

adata.obs['cluster_cellcharter'] = autok.predict(adata, use_rep='X_cellcharter')

adata.write('/data/cephfs-1/home/users/rauertc_c/liposarcoma-wgs/20241104_CC_Xenium_SCVIEpoch/20241106_19Epoch/adata_PostCC_19Epoch.h5ad')

# Plot CellCharter Clusters

sq.pl.spatial_scatter(
    adata,
    color=["cluster_cellcharter"],
    img=None,
    library_key='sample',
    spatial_key="spatial",
    library_id = ['1', '2', '3', '4', '5', '6', '7', '8', '9'],
    size=10,
    ncols=4,
    title=['Region_1', 'Region_2', 'Region_3', 'Region_4', 'Region_5', 'Region_6', 'Region_7', 'Region_8', 'Region_9'], 
    save=f'/data/cephfs-1/home/users/rauertc_c/liposarcoma-wgs/20241104_CC_Xenium_SCVIEpoch/20241106_19Epoch/CellCharter_19Epoch.pdf'
)

plt.close()

# Plot Embedding with CC Cluster
sc.pl.embedding(
    adata,
    basis=SCVI_MDE_KEY,
    color=["sample", "cell_types", "cluster_cellcharter"],
    frameon=False,
    ncols=1,
    legend_loc="center left",
    legend_fontsize='small',
)

# Get all the axes (one for each subplot) and adjust the legends
fig, axes = plt.gcf(), plt.gcf().axes  # Get current figure and axes

# Adjust legend for each subplot
for ax in axes:
    legend = ax.get_legend()
    legend.set_bbox_to_anchor((1.02, 0.5))  # Shift each legend to the right
    legend._legend_box.align = "left"       # Aligns the content of the legend if needed

# Ensure tight layout to avoid clipping
plt.tight_layout()

# Save the figure with all elements included
plt.savefig('/data/cephfs-1/home/users/rauertc_c/liposarcoma-wgs/20241104_CC_Xenium_SCVIEpoch/20241106_800Epoch/SCVI_800_Embedding_CC.pdf', bbox_inches="tight")
plt.close()


In [ ]:
# Visualize only WDLS Cluster

adata_r2=adata[adata.obs['sample'] == "2"].copy()


# run leiden Clustering of Cells

sc.pp.neighbors(adata, use_rep="X_scVI")
sc.tl.leiden(adata)

# Extract the leiden column with identifiers as index
column_to_transfer = adata.obs[['leiden']]  # Extract as DataFrame to keep index

# Save to a CSV file
column_to_transfer.to_csv("/data/cephfs-1/home/users/rauertc_c/liposarcoma-wgs/20241104_CC_Xenium_SCVIEpoch/20241106_19Epoch/leiden_column.csv")

# load into tmux session
# Load the CSV file
loaded_column = pd.read_csv("/data/cephfs-1/home/users/rauertc_c/liposarcoma-wgs/20241104_CC_Xenium_SCVIEpoch/20241106_19Epoch/leiden_column.csv", index_col=0)
adata.obs['leiden'] = adata.obs_names.map(loaded_column['leiden'])
adata.obs['leiden'] = adata.obs["leiden"].astype(str)

# run Embedding without SCVI
adata.obsm["MDE_NoDimRed"] = scvi.model.utils.mde(adata.X, accelerator="cpu")

# Plot Embedding with CC Cluster
sc.pl.embedding(
    adata,
    basis="MDE_NoDimRed",
    color=["leiden"],
    frameon=False,
    ncols=1,
    legend_loc="center left",
    legend_fontsize='small',
)

# Get all the axes (one for each subplot) and adjust the legends
fig, axes = plt.gcf(), plt.gcf().axes  # Get current figure and axes

# Adjust legend for each subplot
for ax in axes:
    legend = ax.get_legend()
    legend.set_bbox_to_anchor((1.02, 0.5))  # Shift each legend to the right
    legend._legend_box.align = "left"       # Aligns the content of the legend if needed

# Ensure tight layout to avoid clipping
plt.tight_layout()

# Save the figure with all elements included
plt.savefig('/data/cephfs-1/home/users/rauertc_c/work/SCVIEpoch/Embedding_CC_NoDimRed_OnlyLeiden.pdf', bbox_inches="tight")
plt.close()

adata.write('/data/cephfs-1/home/users/rauertc_c/work/SCVIEpoch/adata_PostCC_PostMDE_800Epoch.h5ad')

# Plot Celltype Proportions
# Create a dataframe with cluster and cell type information
df = adata.obs[['cluster_cellcharter', 'cell_types']]

# Count cell types per cluster
cell_type_counts = df.groupby(['cluster_cellcharter', 'cell_types']).size().unstack(fill_value=0)

# Calculate proportions by dividing each count by the total count per cluster
cell_type_proportions = cell_type_counts.div(cell_type_counts.sum(axis=1), axis=0)

# Plot
cell_type_proportions.plot(kind='bar', stacked=True, figsize=(10, 6), colormap="tab20")
plt.xlabel("Cluster")
plt.ylabel("Proportion of Cell Types")
plt.title("Cell Type Proportions per Cluster")
plt.legend(title="Cell Type", bbox_to_anchor=(1.05, 1), loc='upper left')

plt.savefig('/data/cephfs-1/home/users/rauertc_c/liposarcoma-wgs/20241104_CC_Xenium_SCVIEpoch/20241106_800Epoch/CC_800_CT_Proportion.pdf', bbox_inches="tight")
plt.close()


# Plot SCVI Embedding with Leiden Cluster
# Plot Embedding with CC Cluster
sc.pl.embedding(
    adata,
    basis=SCVI_MDE_KEY,
    color=["leiden"],
    frameon=False,
    ncols=1,
    legend_loc="center left",
    legend_fontsize='small',
)

# Get all the axes (one for each subplot) and adjust the legends
fig, axes = plt.gcf(), plt.gcf().axes  # Get current figure and axes

# Adjust legend for each subplot
for ax in axes:
    legend = ax.get_legend()
    legend.set_bbox_to_anchor((1.02, 0.5))  # Shift each legend to the right
    legend._legend_box.align = "left"       # Aligns the content of the legend if needed

# Ensure tight layout to avoid clipping
plt.tight_layout()

# Save the figure with all elements included
plt.savefig('/data/cephfs-1/home/users/rauertc_c/work/SCVIEpoch/Embedding_CC_SCVI800_OnlyLeiden.pdf', bbox_inches="tight")
plt.close()
